# MongoDB from Python
This notebook introduces how we can talk to a running instance of `MongoDB` from a python program. Any python program can fetch data from or put data into `MongoDB`--it could be a standalone program like this, an `iPython` shell, or your `Flask` server.

First, we must connect to a running instance of `MongoDB`. Remember that `MongoDB` is just a program that stores and allows effecient querying of data. It can be running anywhere--on your machine (`localhost`), or perhaps a machine in the cloud (`vcm-0000.vm.duke.edu`). We connect to it as follows:

In [1]:
from pymodm import connect

In [2]:
connect("mongodb://localhost:27017/example") # Connect to a mongodb database running 192.168.0.10:27017

The `/example` portion of the connection URI above specifies which MongoDB database or "folder" we want to talk to. Each `MongoDB` instance can have multiple "databases" that are independant of each other. Just think of this as a namespace. If we connect to `example2` instead and it does not exist, a blank database will be created under the namespace `/example2`

## Models (schemas)
MongoDB is very forgiving, and does not _require_ us to specify what collections (tables) we want to store upfront, nor do we have to specify the structure of data we are going to store.

**However**, it is very useful and important to specify some of this structure in advance, so that it is clear in your code what your expectation of data structure is going to be. For example, if we want to store a `User` in the databse, we want it to be very clear in the code what fields a User is going to have and what types each of those fields will be. This allows for validation when storing and retreiving `User`s. 

An example of a "model" or schema definition for our MongoDB interface library (`pymodm`) is below.

In [3]:
from pymodm import MongoModel, fields
class User(MongoModel):
    email = fields.EmailField(primary_key=True)
    first_name = fields.CharField()
    last_name = fields.CharField()
    age = fields.IntegerField()

As you can see, this `User` is just a normal old python class. Since it inherits from `MongoModel` a "super class" it gets many featres for free, including a free initialization method that is based on the fields (like `email`) we specified.

We can use and interact with `User` like a normal python class, see below:

In [4]:
u = User(email="awaxye@yahoo.com", first_name="Adam", last_name="Wax", age="30")
print(u)

<User object>


In [5]:
print(u.email)

awaxye@yahoo.com


In [6]:
print(u.first_name)

Adam


## Save a User
However, this `User` class has some special abilities. For example, if we want to save this `User` `u` to the MongoDB database we connected to we can simply call:

In [7]:
u.save()

User(last_name='Wax', first_name='Adam', email='awaxye@yahoo.com', age=30)

The user is now stored in the MongoDB database!

### Add more Users!
Let's add some more Users to this database:

In [8]:
u2 = User(email="tejank@test.com", first_name="Tejank", last_name="Shah", age="25")
u2.save()
u3 = User(email="dward@test.com", first_name="David", last_name="Ward", age="40")
u3.save()

User(last_name='Wax', first_name='David', email='dward@test.com', age=40)

## Query Users
We can now search for **all** Users in our database as follows.

In [9]:
for user in User.objects.raw({}):
    print(user.email)

awaxye@yahoo.com
dward@test.com
tejank@test.com


As you can see, the `user` variable inside the loop is just an instance of the class `User` we created earlier! We can work with the `user` variable just like we are used to working with classes. We can even modify `user` and then call `user.save()` if we wanted to update the user.

We can also choose to **filter** the `User`s we want to query with conditions like this:

In [10]:
for user in User.objects.raw({"age": 30}):
    print(user.email)

mark@test.com
bob@test.com


As you can see, only `Adam` has an "age" equal to 30, so only that User is fetched to iterate over.

If we expect that a certain query should only return one result, or we just want the first User of a query we can do the following:

In [11]:
tejank_user = User.objects.raw({"first_name": "Tejank"}).first()
tejank_user.first_name

'Tejank'

### Query and Update Users
As mentioned earlier, you can actually fetch a user from the database, update it in python, and call `save()` to update that user in the database. For example if we wanted to update Bob's age:


In [12]:
david_user = User.objects.raw({"first_name": "David"}).first()
david_user.age = 40
david_user.save()

User(last_name='Ward', first_name='David', email='dward@test.com', age=9000)

### Query by primary key

One thing you will notice is that we cannot query users using the email field. This is because we set `email` to be the `primary_key` when we defined our `User` model/schema. When you are looking for a single user, usually you should try to query by whatever the primary_key is.

In [13]:
awaxye = User.objects.raw({"email": "awaxye@yahoo.com"}).first()  # this will NOT work

DoesNotExist: 

Instead we must query primary key fields by the key `"_id"` like so:

In [14]:
awaxye = User.objects.raw({"_id": "awaxye@yahoo.com"}).first()
awaxye.first_name

'Adam'